# Pipeline to Conclusion
> ##          Acquiring Related Market Data --> Analysis --> Inferences --> Expanding Information for Future Use and Consistency


### Web Scrape for the Data

I've used a chrome extension and a little html knowledge to get this data.
My sources consist of
- for the global and Usa market are : builtin.com and 500startups 
- for the Turkey market are : startup borsa and startup market

You can see the related snippets below for imports of used packages and dataframes


In [116]:
#Related Imports
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Turkey Marketplace


In [117]:
df_smarket_hr = pd.read_csv(r'csv/hr_startupmarket.csv')
df_smarket_hr = df_smarket_hr.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1)

df_sborsa_all = pd.read_csv(r'csv/startup_borsa.csv')
df_sborsa_all = df_sborsa_all.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1)

#filtered
df_sborsa_hr = df_sborsa_all[(df_sborsa_all.sector == 'İnsan Kaynakları & İşe Alım')]

#### Global Marketplace includes Turkey


In [ ]:
df_500_hr = pd.read_csv(r'csv/HR_500startup.csv')
df_500_hr = df_500_hr.rename({'name':'title'}, axis=1)
df_500_hr = df_500_hr.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1)

#filtered
df_500_hr_turkey = df_500_hr[(df_500_hr.location == 'Turkey')]

#### USA Marketplace


In [ ]:
df_builtin_hr = pd.read_csv(r'csv/builtin_hr.csv')
df_builtin_hr = df_builtin_hr.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1)

### I have covered the general tasks of HR for subsector classification and how they can be extended. While these tasks vary from source to source, these are the common ones on many:
- Recruiting and staffing employees
- Employee benefits
- Employee compensation
- Employee and labor relations
- Human resources compliance
- Organizational structure
- Human resources information and payroll
- Employee training and development


#### After getting involved with sector columns in our data I can say that

In [82]:
#Subsections on 500Startup
subsections = df_500_hr['sector'].unique()

### Merging subsections with other contexts are : 
- Personnel / Benefits
- Professional training
- College & Continuing Education
- Recruiting
- Tools & Services
- Legal services
- Infrastructure
- Collaboration

### Now having bunch of startups to investigate I need to make selection for which ones to analyze because our time is limited and this part is not automated
#### For making this selection I've looked up the common startups which held in different dataframes.

Technical approach of this task is to use string matcher method like fuzzy. You can see the related code below.

In [119]:
#function for string matching this will help to choose the common titles in datasets so we can find most related ones and lean on them 
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

#### For the Turkey Market I have 3 different combinations to look at we can see the concurrent titles below

In [120]:
s500_match_df = fuzzy_merge(df_smarket_hr, df_500_hr_turkey, 'title', 'title', threshold=80)
s500_match_df[(s500_match_df.matches != '')]

,title,context,link,link-href,logo-src,matches
14,Kolay İK,İşinizi kolaylaştırır.,İncele,https://startupmarket.co/kolay-ik,https://startupmarket.co/cache/100x100/upload/...,KolayIK


Only common startup in here is Kolay IK


In [ ]:
borsa500_match_df = fuzzy_merge(df_500_hr_turkey, df_sborsa_hr, 'title', 'title', threshold=80)
borsa500_match_df[(borsa500_match_df.matches != '')]

As we can see 500 Startups and Startup Borsa do not have any startup in common


In [134]:
marketborsa_match_df = fuzzy_merge(df_sborsa_all, df_smarket_hr, 'title', 'title', threshold=90)
marketborsa_match_df[(marketborsa_match_df.matches != '')]

,title,sector,context,matches
49,SmartCV Teknoloji AŞ,İnsan Kaynakları & İşe Alım,İşe alımda oyunlaştırma teknolojisi ve ML kull...,SmartCV
55,FlexyTime,SaaS,FlexyTime evden veya ofisten verimli çalışması...,FlexyTime
66,İdenfit,Yazılım,"Idenfit; insan kaynakları yönetimine, bütüncül...",Idenfit
125,Yersonel.com,İnsan Kaynakları & İşe Alım,Yersonel.com gastronomi sektörüne odaklanmış k...,Yersonel.com
134,Smarthronline,İnsan Kaynakları & İşe Alım,Online career platform connecting logistics pr...,Smarthronline
206,Verified by Sertifier INC.,Eğitim & Eğitim Teknolojileri,"Verified, Sertifier INC. tarafından geliştiril...",Sertifier
214,ilk-is.com,İnsan Kaynakları & İşe Alım,İş arayan veya iş başvurusu yapmak isteyen her...,ilk-is
237,Ara,Tüketici Hizmetleri,“Türkiye’nin Hizmet Arama Motoru” sloganıyla e...,Zumbara Zaman Kumbarası
254,Test Invite,Eğitim & Eğitim Teknolojileri,"Test Invite, sınav ve ölçme-değerlendirme süre...",Test Invite
267,DinamikCRM,SaaS,DinamikCRM yaklaşık 1 yıl süren ürün hazırlıkl...,DinamikCRM


Except some dummy matches like 
> - ara - zumbara 
> - otherside esports - others

conclusions are satisfactory.

With this merge we can lookup to these startups as a starter for the Turkey Market for the limited time being.
